In [2]:
source("utils/plot.R")

In [83]:
data <- read_tsv(snakemake@input[[1]], col_types=cols()) %>%
    mutate(
        locus_str = paste0('Pos. ', scales::comma(locus))
    ) %>%
    filter(log_prob > quantile(log_prob, 0.75))

In [84]:
selstrens = data %>%
    group_by(locus_str) %>%
    slice_sample(n=1) %>%
    select(locus_str, s_estimate) %>%
    mutate(
        s_str = paste0('s = ', signif(s_estimate))
    )

In [85]:
clues <- ggplot(data) +
    stat_summary_hex(aes(x=gen, y=freq, z=exp(log_prob), alpha=exp(log_prob)), fun='sum', bins=12) +
    geom_label(
        data=selstrens,
        aes(x=max(data$gen), y=max(data$freq), label=s_str),
        size=2.5,
        hjust=0,
        family='Arial Narrow'
    ) +
    facet_wrap(vars(locus_str)) +
    scale_x_reverse() +
    scale_fill_gradient(low='lightgrey', high='black') +
    labs(
        x='Generations ago',
        y='Allele freq.'
    ) +
    turkana_theme +
    theme(
        legend.position = 'none',
        axis.text.x=element_text(angle=45, hjust=1)
    )

turkana_save(snakemake@output[[1]], clues, asp=4/3)